In [ ]:
import numpy as np
import pandas as pd
import itertools
import scipy
from scipy import stats
import statistics
from statsmodels.stats.descriptivestats import sign_test
from statsmodels.stats.weightstats import zconfint
import statsmodels.stats.multitest as smm
from statsmodels.stats.proportion import proportion_confint
from scipy.stats import chi2_contingency

In [ ]:
%pylab inline

In [ ]:
# считываем данные
data = 

In [ ]:
# посмотрим структуру данных
data.head()

# Задание 1

Размер кредитного лимита (LIMIT_BAL). В двух группах, тех людей, кто вернул кредит (default = 0) и тех, кто его не вернул (default = 1) проверьте гипотезы:

a) о равенстве медианных значений кредитного лимита с помощью подходящей интервальной оценки (zconfint)

b) о равенстве распределений с помощью одного из подходящих непараметрических критериев проверки равенства средних. Значимы ли полученные результаты с практической точки зрения ?

In [ ]:
# разбейте людей на 2 группы: default = 0 и default = 1
def_0 = 
def_1 = 

In [ ]:
# построим гистограммы со сглаженной кривой распределения
sns.distplot(def_0['LIMIT_BAL'])
sns.distplot(def_1['LIMIT_BAL'])

In [ ]:
# с помощью zconfint определите доверительный интервал для медианного значения кредитного скоринга

# Вопрос 1. Пересекаются ли интервалы для средних? Что  это значит?

In [ ]:
def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

def get_random_combinations(n1, n2, max_combinations):
    index = list(range(n1 + n2))
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), filter(lambda i: i not in index, range(n))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

pylab.hist(permutation_zero_dist_ind(def_0['LIMIT_BAL'], def_1['LIMIT_BAL'], max_combinations = 1000))
pylab.show()

# Вопрос 2. Опишите своими словами, что делают все три вышеописанные функции.

In [ ]:
def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [ ]:
# проверьте гипотезу о равенстве распределений, где нулевая гипотеза означает их равенство, с помощью permutation_test

# Вопрос 3. Являются ли результаты по кредитным лимитам практически значимыми?

# Задание 2

Пол (SEX): Проверьте гипотезу о том, что гендерный состав группы людей вернувших и не вернувших кредит отличается.

In [ ]:
# разбейте наши подвыборки с default = 0 и default = 1 на мужчин и женщин
# уберите признаки 'SEX', 'default' и 'ID' с помощью drop
male_0 = 
fem_0 =
male_1 = 
fem_1 =

In [ ]:
# постройте с попарно гистограммы для всех признаков у мужчин и женщин с default = 0 
k = 1
for i in fem_0.columns: 
    pylab.subplot(5, 5, k)
    fem_0[i].hist(bins = 20, figsize=(20,20))
    male_0[i].hist(bins = 20, figsize=(20,20))
    k = k + 1

In [ ]:
# построим попарно гистограммы для всех признаков у мужчин и женщин с default = 1 
k = 1
for i in fem_1.columns: 
    pylab.subplot(5, 5, k)
    fem_1[i].hist(bins = 20, figsize=(20,20))
    male_1[i].hist(bins = 20, figsize=(20,20))
    k = k + 1

In [ ]:
# Далее проверим наши выводы численно с помощью тех же перестановок

In [ ]:
p_value_1 = list()
for i in fem_0.columns:
    # оцените p_value для всех столбцов матриц fem_0 и male_0 с помощью permutation_test
    p_value_1.append(p_value)

In [ ]:
p_value_2 = list()
for i in fem_1.columns:
    # оцените p_value для всех столбцов матриц fem_1 и male_1 с помощью permutation_test
    p_value_2.append(p_value)

In [ ]:
# Теперь осуществим поправку на множественность гипотез методом Бенджамини-Хохберга
reject_1, p_corrected_1, a11, a12 = smm.multipletests(p_value_1, alpha=0.025, method='fdr_bh')
reject_2, p_corrected_2, a21, a22 = smm.multipletests(p_value_2, alpha=0.025, method='fdr_bh')

In [ ]:
# найдём количество значимых отличий в группе для fem_0 и male_0
# другими словами, найдите количество p_value меньших 0.025
s_1 = 0
for i in range(len(p_corrected_1)):
    if p_corrected_1[i] < 0.025:
        s_1 = s_1 + 1
s_1

In [1]:
# Сколько признаков из 22 отличаются?

In [ ]:
# найдём количество значимых отличий в группе для fem_1 и male_1
# другими словами, найдите количество p_value меньших 0.025
s_2 = 0
for i in range(len(p_corrected_2)):
    if p_corrected_2[i] < 0.025:
        s_2 = s_2 + 1
s_2

# Вопрос 4. Сколько статистически различимых признаков оказалось в разных группах по половому признаку? Сделайте выводы

# Задание 3

Образование (EDUCATION): Проверьте гипотезу о том, что образование не влияет на то, вернет ли человек долг. Предложите способ наглядного представления разницы в ожидаемых и наблюдаемых значениях количества человек вернувших и не вернувших долг. Например, составьте таблицу сопряженности "образование" на "возврат долга", где значением ячейки была бы разность между наблюдаемым и ожидаемым количеством человек. Как бы вы предложили модифицировать таблицу так, чтобы привести значения ячеек к одному масштабу не потеряв в интерпретируемости ? Наличие какого образования является наилучшим индикатором того, что человек отдаст долг ? наоборт, не отдаст долг ?

EDUCATION: образование (0 = доктор, 1 = магистр; 2 = бакалавр; 3 = выпускник школы; 4 = начальное образование; 5= прочее; 6 = нет данных ).

In [ ]:
# изобразите матрицу сопряжённости "образование" на "возврат долга" с помощью pd.crosstab
confusion_mx = 

In [ ]:
# создадим таблицу согласно распределению хи-квадрат (ожидаемое количество человек)
chi, p, shape, array_chi = chi2_contingency(confusion_mx)

In [ ]:
# заполните таблицу разностями между наблюдаемым и ожидаемым количеством человек

In [ ]:
# приведите таблицу к одному масштабу, записывая в ячейки доли от общего числа
confusion_mx_prop = 

In [ ]:
# создадим таблицу для долей согласно распределению хи-квадрат (ожидаемое количество человек)
chi, p, shape, array_chi_prop = chi2_contingency(confusion_mx_prop)

In [ ]:
# заполните таблицу разностями между наблюдаемыми и ожидаемыми долями

# Вопрос 5. Какие можно сделать выводы?

# Задание 4

Семейное положение (MARRIAGE): Проверьте, как связан семейный статус с индикатором дефолта: нужно предложить меру, по которой можно измерить возможную связь этих переменных и посчитать ее значение.

Посмотрим на пропуски:

In [ ]:
marriage_nan = data[data.MARRIAGE == 0] + data[data.MARRIAGE == 3]

In [ ]:
float(marriage_nan.shape[0]) / data.shape[0]

Доля объектов с неустановленным семейным положением не превышает 1.3%. Восстановить пропущенные данные не представляется возможным. Значит, данные объекты можно удалить

In [ ]:
data_marr_1 = data[data.MARRIAGE == 1]

In [ ]:
data_marr_2 = data[data.MARRIAGE == 2]

In [ ]:
# тереь посчитайте долю холостых людей с дефолтом

In [ ]:
# посчитайте долю женатых/ замужних людей с дефолтом

In [ ]:
# Доля женатых/ замужних людей с дефолтом выше, чем у холостых. Теперь оценим это статистически
# Воспользуемся коэффициентом Крамера. Ничего не трогайте, в ячейке ниже всё готовое.

In [ ]:
def cramers_corrected_stat(confusion_matrix):
    """ calculate Cramers V statistic for categorial-categorial association.
        uses correction from Bergsma and Wicher, 
        Journal of the Korean Statistical Society 42 (2013): 323-328
    """
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))    
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min( (kcorr-1), (rcorr-1)))

In [ ]:
# изобразите матрицу сопряжённости "семейное положение" на "возврат долга" с помощью pd.crosstab
confusion_mx_marr = 

In [ ]:
# посчитаем коэффициент Крамера:
cramers_corrected_stat(confusion_mx_marr)

# Вопрос 6. Какой можно сделать вывод?

# Задание 5

Возраст (AGE): Относительно двух групп людей вернувших и не вернувших кредит проверьте следующие гипотезы:

a) о равенстве медианных значений возрастов людей

b) о равенстве распределений с помощью одного из подходящих непараметрических критериев проверки равенства средних. Значимы ли полученные результаты с практической точки зрения ?

In [ ]:
# разобьём людей на 2 группы: default = 0 и default = 1
def_0 = 
def_1 =

In [ ]:
# построим гистограммы со сглаженной кривой распределения, эту ячейку просто запустите
sns.distplot(def_0['AGE'])
sns.distplot(def_1['AGE'])

In [ ]:
# проверим гипотезу о равенстве медиан с помощью zconfint, как в Задании 1

In [ ]:
# Оцените различия между выборками с помощью критерия Манна-Уитни (stats.mannwhitneyu)


In [ ]:
# проверьте гипотезу о равенстве распределений с помощью перестановочного критерия (permutation_test)

# Вопрос 7. Можно ли отвергать нулевую гипотезу о равенстве медиан? Какой критерий о чём говорит?